In [4]:
import numpy as np
import pandas as pd
from scipy.stats import mode
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve,auc
from imblearn.over_sampling import SMOTE

pd.set_option('display.max_columns',200)
warnings.filterwarnings('ignore')
%matplotlib inline

csv_data=pd.read_csv(r'D:\A\data\2019_xyk.csv')
print(csv_data.shape)
df1=csv_data.dropna(axis=1,how='all')   #删除全是空白值的数据

df2=df1.dropna(thresh=0.7*len(df1),axis=1)   #剔除空白值在70%以上的的列

df3=df2.fillna(df2.mean())    #按照每列的均值填充数据
loan=df3
print(loan.shape)
loan.head()

(43047, 145)
(43047, 102)


,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_inq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,disbursement_method,debt_settlement_flag
0,12000,12000,12000,36 months,6.99%,370.48,A,A2,signmaker/waterjet,10+ years,MORTGAGE,56000,Not Verified,17-Mar,Fully Paid,n,home_improvement,Home improvement,852xx,AZ,17.90,0,Aug-95,0.0,13,0,6665,24%,28,w,0.00,0.00,12773.27240,12773.27,12000.00,773.27,0.0,0.0,0,18-May,8001.3,NaN,18-May,0.0,1,Individual,0,0,139230,0,2,0,3,13.0,14875,63.0,0,0,1835,42.0,27800,2,8,0,4,10710.0,17592.0,20.0,0,0,248.0,259,33,13,5,33.0,17.000000,0,4,7,7,7,9,10,14,7,13,0.0,0,0,0,100.0,0.0,0,0,172161,21540,22000,23500.0,N,Cash,N
1,14000,14000,14000,36 months,8.24%,440.27,B,B1,Retail Sales Consultant,6 years,MORTGAGE,65000,Not Verified,17-Mar,Current,n,home_improvement,Home improvement,329xx,FL,10.80,0,May-99,0.0,5,0,1344,20.70%,8,w,8582.29,8582.29,6591.23000,6591.23,5417.71,1173.52,0.0,0.0,0,18-Jul,440.27,18-Jul,18-Jun,0.0,1,Individual,0,0,169788,1,1,0,0,33.0,18203,61.0,1,1,278,51.0,6500,0,1,0,1,33958.0,722.0,27.8,0,0,121.0,57,0,0,1,57.0,6.104727,0,1,2,1,1,4,2,2,2,5,0.0,0,0,1,100.0,0.0,0,0,197948,20875,1000,29807.0,N,Cash,N
2,35000,35000,35000,60 months,25.49%,1037.38,E,E4,Human Resource Specialist LER,10+ years,MORTGAGE,114000,Source Verified,17-Mar,Fully Paid,n,home_improvement,Home improvement,224xx,VA,14.04,0,Nov-98,1.0,17,0,35694,74.80%,31,w,0.00,0.00,36875.07892,36875.08,35000.00,1875.08,0.0,0.0,0,17-Jun,35565.1,NaN,17-Jun,0.0,1,Individual,0,0,394086,1,2,0,1,19.0,46086,56.0,1,1,10424,70.0,47700,1,3,4,2,24630.0,1481.0,95.5,0,0,157.0,220,0,0,4,26.0,0.000000,4,9,11,10,11,8,14,16,11,17,0.0,0,0,1,87.1,100.0,0,0,424707,81780,32700,52309.0,N,Cash,N
3,11875,11875,11875,36 months,11.44%,391.26,B,B4,Shop foreman,10+ years,MORTGAGE,55000,Not Verified,17-Mar,Current,n,home_improvement,Home improvement,480xx,MI,34.50,0,Apr-97,0.0,14,0,17823,39.40%,28,w,7414.06,7414.06,5853.81000,5853.81,4460.94,1392.87,0.0,0.0,0,18-Jul,391.26,18-Jul,18-Jun,0.0,1,Individual,0,0,241115,0,3,0,2,17.0,43814,78.0,0,2,10023,61.0,45200,1,3,2,4,17223.0,7934.0,67.3,0,0,239.0,191,13,13,2,46.0,9.000000,0,3,6,4,5,14,10,12,6,14,0.0,0,0,0,96.4,25.0,0,0,319205,61637,24300,56205.0,N,Cash,N
4,1500,1500,1500,36 months,5.32%,45.18,A,A1,Auto Technician,3 years,MORTGAGE,67000,Not Verified,17-Mar,Current,n,car,Car financing,593xx,MT,5.95,1,1-Dec,0.0,5,0,8673,43.80%,16,w,903.77,903.77,676.81000,676.81,596.23,80.58,0.0,0.0,0,18-Jul,45.18,18-Jul,18-Jun,1.0,1,Individual,0,0,142464,0,1,0,1,23.0,1985,38.0,0,0,5044,43.0,19800,0,1,0,1,28493.0,5327.0,62.0,1,0,183.0,127,36,23,3,41.0,18.000000,1,2,2,2,5,6,3,7,2,5,0.0,0,1,0,93.8,0.0,0,0,163700,10658,14000,5200.0,N,Cash,N


In [6]:
loan_status=pd.DataFrame({'意思':['还款中','审核通过','全额结清','宽限期','逾期（31-120天）','逾期（16-30天）','坏账','违约'],'数量':loan['loan_status'].value_counts().values},
                        index=loan['loan_status'].value_counts().index)
loan_status

,意思,数量
Current,还款中,32484
Fully Paid,审核通过,7940
Charged Off,全额结清,1303
Late (31-120 days),宽限期,703
In Grace Period,逾期（31-120天）,365
Late (16-30 days),逾期（16-30天）,199
Default,坏账,52
n,违约,1


In [7]:
# Y值处理
loan['loan_status']=loan['loan_status'].replace(['Fully Paid','In Grace Period','Late (31-120 days)','Late (16-30 days)','Charged Off','Default'],
                                               ['0','1','1','1','1','1'])
loan=loan[loan['loan_status'].isin(['0','1'])]
loan['loan_status']=loan['loan_status'].astype('int')

In [8]:
# 缺失值处理
null_cols=loan.isna().sum().sort_values(ascending=False)/float(loan.shape[0])
null_cols[null_cols > .3]

loan=loan.dropna(thresh=loan.shape[0]*.7,axis=1)
names=['sub_grade','emp_title','pymnt_plan','title','zip_code','total_rec_late_fee','recoveries','collection_recovery_fee','last_pymnt_d','last_pymnt_amnt',
      'last_credit_pull_d','collections_12_mths_ex_med','policy_code','acc_now_delinq','num_tl_120dpd_2m','num_tl_30dpd','hardship_flag','debt_settlement_flag',
      'funded_amnt','funded_amnt_inv','out_prncp_inv','total_pymnt','total_pymnt_inv','out_prncp','total_rec_prncp','total_rec_int']
loan=loan.drop(names,axis=1)
loan=loan.drop(loan[loan['earliest_cr_line'] == '1900/1/0'].index)

In [10]:
# 对缺失值进行插补
loan['int_rate']=loan['int_rate'].str.replace('%','').astype('float')
loan['revol_util']=loan['revol_util'].str.replace('%','').astype('float')

loan['issue_d']=pd.to_datetime(loan['issue_d'],format='%d-%m')
loan['earliest_cr_line']=pd.to_datetime(loan['earliest_cr_line'],format='%Y/%m/%d')
loan['mth_interval']=loan['issue_d']-loan['earliest_cr_line']
loan['mth_interval']=loan['mth_interval'].apply(lambda x: round(x.days/30,0))
loan['issue_m']=loan['issue_d'].apply(lambda x: x.month)
loan=loan.drop(['issue_d','earliest_cr_line'],axis=1)

loan['emp_length']=loan['emp_length'].fillna('0')
loan['emp_length']=loan['emp_length'].str.replace(' year','')
loan['emp_length']=loan['emp_length'].str.replace('s','')
loan['emp_length']=loan['emp_length'].replace(['10+','< 1'],['11','0']).astype('int')

loan['income_vs_loan']=loan['annual_inc']/loan['loan_amnt']
loan['delinq_2yrs']=loan['delinq_2yrs'].astype('int')
loan['revol_bal']=loan['revol_bal'].astype('int')
loan['total_acc']=loan['total_acc'].astype('int')

AttributeError: Can only use .str accessor with string values, which use np.object_ dtype in pandas